## Download the dataset - from Kaggle

Skip this step if you have the data in Drive already!

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

{}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
! kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [6]:
! kaggle competitions download -c bgu-recommendation-systems-click-through-rate

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [8]:
import zipfile
import os

zip_file_path = 'bgu-recommendation-systems-click-through-rate.zip'
extracted_folder_path = 'bgu-recommendation-systems-click-through-rate'

# Create the extraction directory if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Open the zip file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"Successfully extracted contents to {extracted_folder_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'bgu-recommendation-systems-click-through-rate.zip'

## Download the dataset - from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

parent_dir = '/content/drive/MyDrive/cs247project/data/train_data'

# Initialize an empty DataFrame to store the combined data
df = pd.DataFrame()

# Iterate through each folder (train1 through train7)
for folder_name in os.listdir(parent_dir):
    folder_path = os.path.join(parent_dir, folder_name)

    # Check if the path is a directory
    if os.path.isdir(folder_path):
        # Iterate through each CSV file in the folder
        for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {folder_name}"):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)

                # Read the CSV file into a DataFrame
                this_df = pd.read_csv(file_path)

                # Concatenate the current DataFrame with the combined_data DataFrame
                df = pd.concat([df, this_df], ignore_index=True)

# Print the combined DataFrame
print(df)

Processing train4:  70%|███████   | 7/10 [01:45<00:48, 16.28s/it]

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

## Get basic info about features

In [2]:
df.head(5)

NameError: name 'df' is not defined

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.count()

In [ ]:
# Visualize is_click, make sure the classes are balanced
plt.figure(figsize=(8, 5))
sns.countplot(x='is_click', data=df)
plt.title('Distribution of Clicks')
plt.xlabel('is_click')
plt.ylabel('Count')
plt.show()

## Remove null values

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.dropna()    # Dropping the missing values.
df.count()

In [ ]:
print(df.isnull().sum())

## Check for outliers

In [ ]:
cols = ['page_view_start_time', 'empiric_clicks', 'user_recs', 'time_of_day'] # can add/remove from this as needed

for col in cols:
      # Create a boxplot for the current column
      sns.boxplot(x=df[col])

      # Set plot title with the column name
      plt.title(f'Boxplot for {col}')

      # Show the plot
      plt.show()

In [ ]:
unique_user_ids = df['user_id_hash'].nunique()
print(unique_user_ids)

In [ ]:
target_id_hash = df['target_id_hash'].nunique()
print(target_id_hash)

## Identify most important features:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
df_numeric_features_only = df.loc[:, numeric_features]

X = df_numeric_features_only.drop('is_click', axis=1)
y = df_numeric_features_only['is_click']

# Create and train the RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Get feature importances
feature_importances = model.feature_importances_

# Combine with feature names
feature_importance_dict = dict(zip(X.columns, feature_importances))

# Sort and display
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_features)

In [ ]:
# Plot the two most important features
plt.figure(figsize=(10, 6))

# Scatter plot with custom colors based on is_click
colors = df['is_click'].map({0: 'red', 1: 'green'})
plt.scatter(df['empiric_calibrated_recs'], df['empiric_clicks'], c=colors)

# Adding labels and title
plt.title('Scatter Plot: empiric_calibrated_recs vs empiric_clicks')
plt.xlabel('empiric_calibrated_recs')
plt.ylabel('empiric_clicks')

plt.show()

In [ ]:
# Plot the some of the less important features
plt.figure(figsize=(10, 6))

# Scatter plot with custom colors based on is_click
colors = df['is_click'].map({0: 'red', 1: 'green'})
plt.scatter(df['user_target_recs'], df['user_recs'], c=colors)

# Adding labels and title
plt.title('Scatter Plot: user_target_recs vs user_recs')
plt.xlabel('user_target_recs')
plt.ylabel('user_recs')

plt.show()

Reference: https://www.kaggle.com/code/imoore/intro-to-exploratory-data-analysis-eda-in-python

## Baseline Classifiers

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# All our baselines only use the numerical features in the dataset
X = df_numeric_features_only.drop('is_click', axis=1)  # Features
y = df_numeric_features_only['is_click']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
model = LogisticRegression(random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training set
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Create a Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Train the classifier on the training set
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")


### Perceptron

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define one-layer neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X_train_scaled.shape[1],))
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)